<a href="https://colab.research.google.com/github/aSafarpoor/Seminar/blob/main/shortest_path.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tqdm import tqdm
import numpy as np
import random

In [ ]:
!wget https://raw.githubusercontent.com/aSafarpoor/Seminar/main/Dataset/train.txt

--2022-05-28 10:10:51--  https://raw.githubusercontent.com/aSafarpoor/Seminar/main/Dataset/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3135377 (3.0M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]   2.99M  --.-KB/s    in 0.07s   

2022-05-28 10:10:52 (44.6 MB/s) - ‘train.txt’ saved [3135377/3135377]



# Graph Preparetion

In [ ]:
f = open("train.txt", "r")
rawdata = f.read()

In [ ]:
rawdata

'00260881\t_hypernym\t00260622\n01332730\t_derivationally_related_form\t03122748\n06066555\t_derivationally_related_form\t00645415\n09322930\t_instance_hypernym\t09360122\n07193596\t_derivationally_related_form\t00784342\n01768969\t_derivationally_related_form\t02636811\n01455754\t_hypernym\t01974062\n07554856\t_hypernym\t07553301\n00057306\t_hypernym\t00056912\n10341660\t_derivationally_related_form\t02661252\n13219258\t_hypernym\t13167078\n01698271\t_also_see\t01754576\n10499355\t_hypernym\t10083823\n02103406\t_hypernym\t02084071\n07190941\t_hypernym\t07185325\n12090318\t_member_meronym\t12093769\n12213635\t_member_meronym\t12214245\n02651424\t_derivationally_related_form\t02672371\n10668450\t_hypernym\t10525134\n01837746\t_hypernym\t01342529\n04692908\t_derivationally_related_form\t01259005\n00848169\t_derivationally_related_form\t06719579\n10483138\t_derivationally_related_form\t00968211\n03631445\t_derivationally_related_form\t01521603\n08135342\t_hypernym\t08123167\n12219065\t_hy

In [ ]:
rawdata = rawdata.split('\n')
dataset = [row.split('\t') for row in rawdata][:-1]

In [ ]:
def generate_graph(dataset):
    node_list = []
    edge_list = []
    neighborhood = {}

    for element in tqdm(dataset):
        head = element[0]
        tail = element[2]
        relation = element[1]
        if relation not in edge_list:
            edge_list.append(relation)
        if head not in node_list:
            node_list.append(head)
        if tail not in node_list:
            node_list.append(tail)
        try:
            neighborhood[head].append((tail,relation))
        except:
            neighborhood[head]=(tail,relation)
        try:    
            neighborhood[tail].append((head,relation))
        except:
            neighborhood[tail]=(head,relation)

    return node_list,edge_list,neighborhood

In [ ]:
node_list,edge_list,neighborhood = generate_graph(dataset)

100%|██████████| 86835/86835 [01:00<00:00, 1436.71it/s]


# shortest path, finding negative samples

In [ ]:
def find_neg_samples(graph,th,node,node_list,num_of_neigh):
    depth = 0
    graph = graph.copy()
    not_visited_nodes = node_list.copy()
    neg_list = []
    bfs_list = [(node,0)]
    most_d = 0
    while len(bfs_list)>0:
        target = bfs_list[0]
        try:
            bfs_list = bfs_list[1:]
        except:
            bfs_list = []
        for neigh in graph[target[0]]:
            if neigh in not_visited_nodes:
                bfs_list.append((neigh,target[1]+1))
                most_d = target[1]+1
                not_visited_nodes.remove(neigh)
                if target[1] >= th:
                    neg_list.append(neigh)
            if len(neg_list)>=num_of_neigh:
                break
    if most_d>2:
        print(most_d,len(neg_list))

    if len(neg_list)>=num_of_neigh:
        return neg_list[:num_of_neigh]
    while len(neg_list)<num_of_neigh:
        secure_random = random.SystemRandom()
        member = secure_random.choice(not_visited_nodes)
        if member not in neg_list:
            neg_list.append(member)
    return neg_list[:num_of_neigh]

In [ ]:
graph = neighborhood.copy()

In [ ]:
for i in tqdm(range(100)):
    find_neg_samples(graph,th=2,node=node_list[i],node_list=node_list,num_of_neigh=5)


 24%|██▍       | 24/100 [00:00<00:00, 116.19it/s]

2
3
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


 49%|████▉     | 49/100 [00:00<00:00, 118.56it/s]

3
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


 73%|███████▎  | 73/100 [00:00<00:00, 112.64it/s]

2
2
2
2
2
2
2
2
3
1
3
1
2
2
2
3
1
2
2
2
2
2
2
2
2
2
2


 85%|████████▌ | 85/100 [00:00<00:00, 106.87it/s]

2
3
1
2
3
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


100%|██████████| 100/100 [00:00<00:00, 108.31it/s]

2
2
2
2
2


In [ ]:
for i in graph['01967373']:
    print(i)

07434209
_derivationally_related_form
